In [1]:
import os
import pandas as pd

input_folder = '$HOME/Projects/raceup/2018-19/telemetry/_data/2019-04-14/merged_imu/'

In [3]:
def find_files(folder, extension='.CSV'):
    """
    :param folder: folder containing files to be merged
    :param extension: allow only this extension
    :return: assuming a sorted order of files, this returns list of files to
    be merged
    """

    files_in_folder = [
        os.path.join(folder, f)  # full path
        for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f)) and f.endswith(extension)
    ]
    return sorted(files_in_folder)

In [4]:
files = find_files(input_folder)

In [53]:
time_key = 'arduino time'
dfs = [
    pd.read_csv(f, skiprows=1)  # first row is version of Arduino
    for f in files
]

In [54]:
time_so_far = 0
for df in dfs:
    try:
        diff_time = df[time_key].iloc[-1]  # last value
    except:
        diff_time = 0

    df[time_key] = df[time_key].apply(lambda x: x + time_so_far)  # add time of previous df
    time_so_far += diff_time

In [58]:
merged = pd.concat(dfs)

In [60]:
merged.to_csv('merged.csv', index=False)

In [1]:
lines = [1503, 5003, 1003, 503, 3003, 4003, 2003, 1753, 753, 3253, 9253, 3, 1003, 1253, 1503, 503, 15753, 2003, 1253, 3253, 11003]
tot_lines = sum(lines)
tot_lines - 69522
len(lines)

21

In [67]:
def without_duplicates(lst):
    return list(set(lst))

In [69]:
def interpol(first_point, second_point):
    """ linear interpol """

    delta_x = second_point[0] - first_point[0]
    delta_y = second_point[1] - first_point[1]
    f = lambda x: (x - first_point[0]) * (delta_y / delta_x) + first_point[1]
    return f

In [ ]:
def add_missing_points(df, eval_points, interpol_f, dnf_val=0.0):
    new_df = []
    ref_key = 'x'
    data_key = 'y'
    
    i = -1
    for point in eval_points:
        new_row = [point]

        if i < len(df[ref_key]): and df[ref_key].iloc[i + 1] == point:  # there is a value at the exact key
            i += 1
            exact_value = df[data_key].iloc[i]
            new_row.append(exact_value)
        else:
            if i < 0:  # no data found ever
                new_row.append(dnf_value)  # dnf
            else:  # data is to be interpolated
                if i < len(df[ref_key]) - 1:
                    prev_data_point = (df1[join_key].iloc[i], df1[data_key].iloc[i])  # (x, y)
                    next_data_point = (df1[join_key].iloc[i + 1], df1[data_key].iloc[i + 1])  # (x, y)
                    interpol_f = interpol(prev_data_point, next_data_point)  # get interpol function
                    interpol_value = interpol_f(index)  # evaluate at missing data point
                    new_row.append(interpol_value)
                else:  # enf of df
                    new_row.append(dnf_value)

In [66]:
df1 = pd.DataFrame([
    [200, 3],
    [210, 4],
    [220, 5],
    [230, 6],
    [240, 1],
], columns=['x', 'y']
)

df2 = pd.DataFrame([
    [180, 43],
    [190, 44],
    [211, 45],
    [225, 46],
    [250, 41],
], columns=['x', 'y']
)

join_key = 'x'
data_key = 'y'
dnf_value = 0.0
indexes = sort(without_duplicates(list(df1[join_key]) + list(df2[join_key])))

i = -1  # start at 0
j = -1
new_df = []

for index in indexes:
    new_row = [i]

    if df1[join_key].iloc[i + 1] == index:  # there is a value at the exact key
        exact_value = df1[data_key].iloc[i]
        new_row.append(exact_value)
        i += 1
    else:
        if i < 0:  # no data found ever
            new_row.append(dnf_value)  # dnf
        else:  # data is to be interpolated
            prev_data_point = (df1[join_key].iloc[i], df1[data_key].iloc[i])  # (x, y)
            next_data_point = (df1[join_key].iloc[i + 1], df1[data_key].iloc[i + 1])  # (x, y)
            interpol_f = interpol(prev_data_point, next_data_point)  # get interpol function
            interpol_value = interpol_f(index)  # evaluate at missing data point
            new_row.append(interpol_value)
            
    
    if df2[join_key].iloc[j] == index:  # there is a value at the exact key
        exact_value = df2[data_key].iloc[j]
        new_row.append(exact_value)
        j += 1

In [70]:
def get_last_time(f):
    with open(f, 'r') as reader:
        try:
            last_line = reader.readlines()[-1]
            last_time = last_line.split(',')[0]
            return float(last_time)
        except:
            return 0

In [72]:
def min_to_str(x):
    dec_secs = x % 1  # float part
    secs = dec_secs * 60
    mins = math.floor(x)  # int part
    return {'min': mins, 'secs': secs}